In [1]:
import pyspark
import pandas as pd

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql import types
from pyspark.sql.functions import *

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('homework_5') \
    .getOrCreate()


22/03/07 14:40:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
#Question 1
spark.version

'3.0.3'

In [5]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-02.csv')

In [6]:
fhvhv_schema = types.StructType([
    types.StructField("hvfhs_license_num", types.StringType(),True),
    types.StructField("dispatching_base_num", types.StringType(),True),
    types.StructField("pickup_datetime", types.TimestampType(),True),
    types.StructField("dropoff_datetime", types.TimestampType(),True),
    types.StructField("PULocationID", types.IntegerType(),True),
    types.StructField("DOLocationID", types.IntegerType(),True),
    types.StructField("SR_Flag", types.StringType(),True)])

In [7]:
df = spark.read \
    .option("header", "true") \
    .schema(fhvhv_schema) \
    .csv('fhvhv_tripdata_2021-02.csv')


In [8]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02764|2021-02-01 00:10:40|2021-02-01 00:21:09|          35|          39|   null|
|           HV0003|              B02764|2021-02-01 00:27:23|2021-02-01 00:44:01|          39|          35|   null|
|           HV0005|              B02510|2021-02-01 00:28:38|2021-02-01 00:38:27|          39|          91|   null|
|           HV0005|              B02510|2021-02-01 00:43:37|2021-02-01 01:23:20|          91|         228|   null|
|           HV0003|              B02872|2021-02-01 00:08:42|2021-02-01 00:17:57|         126|         250|   null|
|           HV0003|              B02872|2021-02-01 00:26:02|2021-02-01 00:42:51|

In [ ]:
df.repartition(24).\
    write.parquet('data/pq/fhvhv/2021/02/')


In [15]:
#Question 2
!ls -lh data/pq/fhvhv/2021/02/ | grep total

total 208M


### Reading/Working with the parquet file

In [16]:
df_fhvhv = spark.read.parquet('data/pq/fhvhv/2021/02/')

In [17]:
df_fhvhv.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02887|2021-02-06 01:18:35|2021-02-06 01:40:34|         163|         235|   null|
|           HV0005|              B02510|2021-02-05 07:13:06|2021-02-05 07:31:56|         225|         181|   null|
|           HV0003|              B02869|2021-02-04 16:56:52|2021-02-04 17:21:36|         260|          95|   null|
|           HV0003|              B02871|2021-02-03 18:34:17|2021-02-03 18:57:12|         235|          60|   null|
|           HV0003|              B02869|2021-02-04 07:25:09|2021-02-04 07:30:34|          55|          55|   null|
|           HV0003|              B02836|2021-02-04 23:15:27|2021-02-04 23:34:29|

In [18]:
df_fhvhv.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [19]:
#df_fhvhv.registerTempTable('fhvhv_trips')
df_fhvhv.withColumn("Trip_Duration_in_Seconds", col("dropoff_datetime").cast("long") - 
        col("pickup_datetime").cast("long")).registerTempTable('fhvhv_trips')

In [20]:
#Question 3
spark.sql("""
SELECT COUNT (*) FROM fhvhv_trips
WHERE date_format(pickup_datetime, 'dd')=15;

""").show()

+--------+
|count(1)|
+--------+
|  367170|
+--------+



In [21]:
spark.sql("""
    SELECT MAX(Trip_Duration_in_Seconds) FROM fhvhv_trips; """).show()

+-----------------------------+
|max(Trip_Duration_in_Seconds)|
+-----------------------------+
|                        75540|
+-----------------------------+



In [22]:
#Question 4
spark.sql("""

SELECT 
    date_format(pickup_datetime, 'dd')
FROM 
    fhvhv_trips
WHERE
    Trip_Duration_in_Seconds = (SELECT MAX(Trip_Duration_in_Seconds) FROM fhvhv_trips);

""").show()

+--------------------------------+
|date_format(pickup_datetime, dd)|
+--------------------------------+
|                              11|
+--------------------------------+



In [26]:
#Question 5
spark.sql("""
   
SELECT 
    dispatching_base_num, COUNT(*)
FROM 
    fhvhv_trips
GROUP BY
    dispatching_base_num
ORDER BY
    count(1)
DESC
LIMIT 
    1;

""").show()

+--------------------+--------+
|dispatching_base_num|count(1)|
+--------------------+--------+
|              B02510| 3233664|
+--------------------+--------+



In [24]:
df_zones = spark.read.option("header", "true").csv('taxi+_zone_lookup.csv')
df_zones.show()
df_zones.registerTempTable('trip_zones')

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [25]:
#Question 6
spark.sql("""
   
SELECT 
   b.zone as PU_zone, c.zone as DO_zone, COUNT(*) trips_total
FROM 
    fhvhv_trips AS a
    
INNER JOIN
    trip_zones AS b
ON
    a.PULocationID = b.LocationID
INNER JOIN
    trip_zones AS c
ON
    a.DOLocationID = c.LocationID

GROUP BY 
    PU_zone, DO_zone
ORDER BY
    COUNT(1)
DESC
LIMIT 
    10;

""").show()

+--------------------+--------------------+-----------+
|             PU_zone|             DO_zone|trips_total|
+--------------------+--------------------+-----------+
|       East New York|       East New York|      45041|
|        Borough Park|        Borough Park|      37329|
|            Canarsie|            Canarsie|      28026|
| Crown Heights North| Crown Heights North|      25976|
|           Bay Ridge|           Bay Ridge|      17934|
|     Jackson Heights|     Jackson Heights|      14688|
|             Astoria|             Astoria|      14688|
|Central Harlem North|Central Harlem North|      14481|
|      Bushwick South|      Bushwick South|      14424|
|Flatbush/Ditmas Park|Flatbush/Ditmas Park|      13976|
+--------------------+--------------------+-----------+

